In [1]:
import sys
sys.path.insert(1,'./Instances_decoders')
sys.path.insert(1,'./Instances_models')
sys.path.insert(1,'./Evaluators')
from TSCFLP_greedy_decoder import greedy_decoder
from TSCFLP_instance import TSCFLPinstance
from evaluators import evaluator_Best_Mean
import torch
import torch.nn as nn
import numpy as np

ImportError: cannot import name 'evaluator_Best_Mean' from 'evaluators' (C:\Users\Felipe\Projetos\Generative models optimization\Generative-model-optimization\./Evaluators\evaluators.py)

In [ ]:
class Generator(nn.Module):
    def __init__(self,I,J,population):
        super().__init__()
        self.population = population
        self.rede = nn.Sequential(
            nn.Linear(I+J,200),
            nn.ReLU(),
            nn.Linear(200,300),
            nn.ReLU(),
            nn.Linear(300,300),
            nn.ReLU(),
            nn.Linear(300,(I+J)*population),
            nn.Sigmoid()
        )
    def forward(self,ruido):
        x = self.rede(ruido)
        x = torch.reshape(x,(self.population,150))
        return x
        
        

In [ ]:
model = Generator(50,100,200)
instancia = TSCFLPinstance(r"C:\Users\Felipe\Projetos\TSCFLP\Instances\PSC1-C1-50.txt")
decoder = greedy_decoder(instancia)
evaluator = evaluator_Best_Mean(decoder)
lr = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr=lr)
Epochs = 300

In [ ]:
def train_loop(model,evaluator,optimizer,Epochs):
    epochs_loss = []
    
    
    for epoch in range(Epochs):
        ruido = torch.randn(150)
        population = model(ruido)
        
  
        optimizer.zero_grad()
        loss = evaluator(population)
        loss.requires_grad_()
        
        loss.backward()
        
        optimizer.step()
        epochs_loss.append(loss.item())
        
        if epoch%10==0:
            print(f'Melhor individuo:{evaluator.best_fit} Média da população: {evaluator.mean_current_pop}')
    return epochs_loss,evaluator